In [1]:
from itertools import repeat
from ast import literal_eval
from pathlib import Path
import json
import pandas as pd

In [2]:
import gzip
import pickle
from wyckoff_transformer.evaluation import StatisticalEvaluator
with gzip.open(Path("cache", "mp_20_biternary", "data.pkl.gz"), "rb") as f:
    datasets_pd = pickle.load(f)
evaluator = StatisticalEvaluator(datasets_pd["test"], pd.concat([datasets_pd["train"], datasets_pd["val"]], axis=0))

In [3]:
GENERATED_PATH = Path("generated")
wycryst_data_raw = pd.read_csv(GENERATED_PATH / "WyCryst_mp_20_biternary.csv", index_col=0,
                            converters=dict(zip(
                                ("reconstructed_ratio1", "reconstructed_wyckoff", "str_wyckoff", "ter_sys"),
                                repeat(literal_eval, 4)
                            )))

In [4]:
from wyckoff_transformer.evaluation import wycryst_to_pyxtal_dict

def check_novelty_wycryst(record):
    pyxtal_dict = wycryst_to_pyxtal_dict(record)
    if pyxtal_dict is None:
        return False
    return evaluator.generated_to_fingerprint(pyxtal_dict) not in evaluator.train_fingerprints
novel_wycryst_pd = wycryst_data_raw[wycryst_data_raw.apply(check_novelty_wycryst, axis=1)]
novel_wycryst_pd.to_csv(GENERATED_PATH / "WyCryst_novel_mp_20_biternary.csv.gz", compression={'method': 'gzip', 'mtime': 0})

ValueError: If mapping, compression must have key 'method'

In [ ]:
generated_wycryst = wycryst_data_raw.apply(wycryst_to_pyxtal_dict, axis=1).dropna()

In [ ]:
with gzip.open(GENERATED_PATH / "WyckoffTransformer_mp_20_biternary.json.gz", "rt") as f:
    generated_wyckoff_transformer = json.load(f)

In [ ]:
import pickle
from wyckoff_transformer.evaluation import (
    timed_smact_validity_from_record, StatisticalEvaluator,
    smac_validity_from_counter, generated_to_fingerprint)

print("Test novelty: ",evaluator.get_test_novelty())
print("WyCryst novelty: ", evaluator.count_novel(generated_wycryst))
print("WyckoffTransformer novelty: ", evaluator.count_novel(generated_wyckoff_transformer))
novel_wycryst = evaluator.get_novel(generated_wycryst)
novel_wyckoff_transformer = evaluator.get_novel(generated_wyckoff_transformer)

Test novelty:  0.9598201742062377
WyCryst novelty:  0.29177947812438687
WyckoffTransformer novelty:  0.8648


In [ ]:
from utils import dump_json_to_gz
dump_json_to_gz(GENERATED_PATH / "WyckoffTransformer_novel_mp_20_biternary.json.gz", novel_wyckoff_transformer)

In [ ]:
novel_evaluator = StatisticalEvaluator(evaluator.get_novel_dataframe(datasets_pd["test"]),
                                       pd.concat([datasets_pd["train"], datasets_pd["val"]], axis=0))

In [10]:
smact_validity_test = sum(map(smac_validity_from_counter, datasets_pd["test"]["composition"]))/len(datasets_pd["test"])
print("Test SMAC-T: ", smact_validity_test)

datasets = {
    "WyCryst": (generated_wycryst, evaluator),
    "WyckoffTransformer": (generated_wyckoff_transformer, evaluator),
    "WyCryst Novel": (novel_wycryst, novel_evaluator),
    "WyckoffTransformer Novel": (novel_wyckoff_transformer, novel_evaluator)}
for name, (dataset, this_evaluator) in datasets.items():
    print(name)
    smact_validity = sum(map(timed_smact_validity_from_record, dataset))/len(dataset)
    print("SMAC-T validity: ", smact_validity)
    print("Num sites KS: ", this_evaluator.get_num_sites_ks(dataset))
    print("Num elements KS :", this_evaluator.get_num_elements_ks(dataset))
    print("DoF KS: ", this_evaluator.get_dof_ks(dataset))
    sg_chi2 = this_evaluator.get_sg_chi2(dataset, sample_size="test")
    print(f"SG chi2: statistic={sg_chi2.statistic}, pvalue={sg_chi2.pvalue}")

Test SMAC-T:  0.8981455465018263
WyCryst
SMAC-T validity:  0.8373454973513832
Num sites KS:  KstestResult(statistic=0.03975535196011648, pvalue=1.4236638119437514e-09, statistic_location=4, statistic_sign=1)
Num elements KS : KstestResult(statistic=0.03838199508352241, pvalue=5.947351369026711e-09, statistic_location=2, statistic_sign=-1)
DoF KS:  KstestResult(statistic=0.04117394256950424, pvalue=3.0839361418463316e-10, statistic_location=7, statistic_sign=1)
SG chi2: statistic=703.5853771023179, pvalue=1.6072519094623623e-73
WyckoffTransformer


SMAC-T validity:  0.8063
Num sites KS:  KstestResult(statistic=0.02380339983141332, pvalue=0.017604870205337915, statistic_location=4, statistic_sign=1)
Num elements KS : KstestResult(statistic=0.0257, pvalue=0.008049610835145911, statistic_location=3, statistic_sign=1)
DoF KS:  KstestResult(statistic=0.0188548468670975, pvalue=0.10231187840614872, statistic_location=2, statistic_sign=1)
SG chi2: statistic=152.11975499959556, pvalue=0.3691994206142184
WyCryst Novel
SMAC-T validity:  0.7335597095212479
Num sites KS:  KstestResult(statistic=0.2796992422819146, pvalue=0.0, statistic_location=3, statistic_sign=1)
Num elements KS : KstestResult(statistic=0.004451737339108447, pvalue=0.9998335648003727, statistic_location=2, statistic_sign=1)


In [ ]:
train_sg = set(datasets_pd["train"].spacegroup_number)
train_sg.update(datasets_pd["val"].spacegroup_number)
test_sg = frozenset(datasets_pd["test"].spacegroup_number)

In [ ]:
len(train_sg), len(test_sg), len(train_sg & test_sg)

(169, 148, 145)